# 2X - 1 Argument

At first glance:  2X+1 looks like a much better way of looking at odds in the Collatz problem than 2X-1

Show that for any Collatz Matrix and Y vector that gives a Collatz Chain X:

$$
A=\begin{pmatrix}⋱&⋮&⋮&⋮\\…&-1&2&0\\…&0&-3&2\\…&1&0&-1\end{pmatrix}
$$

and

$$
Y=\begin{pmatrix}…&0&1&0\end{pmatrix}
$$

That if we create the equation:

$$
A_{2n+1}=\begin{pmatrix}-1&2&0&0&0\\0&⋱&⋮&⋮&⋮\\0&…&-1&2&0\\0&…&0&-3&2\\0&…&1&0&-1\end{pmatrix}
$$

and

$$
Y_{2n+1}=\begin{pmatrix}1&…&0&1&0\end{pmatrix}
$$

To get a new $X_{2n+1}[0]\ = \ 2X[0] - 1$

We can find an $A'_{2n-1}$ and $Y'_{2n-1}$ that follow the Collatz Form

NOTE: This new row is like an Up operation except instead of $Y=0$ we have $Y=1$

## Is there any way to know if the new matrix needs to have a larger or smaller rank than the current?

I would think it would be easier to rule out a smaller ranked matrix before creating a larger one.

## We need an algorithm for finding the Up and Down operations that extend the matrix

### Lets look at $26\ \rightarrow \ 51$
Want an even that comes from an odd

In [1]:
import sys, io
import math
import numpy as np
import pandas as pd
from scipy.optimize import nnls
from fractions import Fraction
from sympy import factorint

In [3]:
def collatzChain(collatzNumber):
    chain = [collatzNumber]
    while collatzNumber != 1:
        if collatzNumber & 1 == 0:
            collatzNumber = collatzNumber // 2
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
        chain.append(collatzNumber)
    return chain
#
def ChainPath(collatzNumber):
    path = []
    while collatzNumber != 1:
        if collatzNumber & 1 == 0:
            collatzNumber = collatzNumber // 2
            path.append("1")
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
            path.append("0")
    return "".join(path)
#
def fractionFromNodeTup(tup):
    p2, p3, c = tup
    fract = Fraction(2**p2 - c, 3**p3)
    return (fract.numerator, fract.denominator)
#

def TupChainFromPath(chain_path):
    tup_chain = [(0, 0, 0)]
    for chain_item in chain_path:
        p2, p3, c = tup_chain[-1]
        if chain_item == "1":
            tup_chain. append((p2 + 1, p3, c))
        else:
            tup_chain. append((p2 + 1, p3 + 1, c*3 + 2**p2))
            
        fract = fractionFromNodeTup(tup_chain[-1])
    return tup_chain
#
def downUpTup(tup):
    p2, p3, c = tup
    p2_01, p3_01, c_01 = (p2 + 2, p3 + 1, c*3 + 2**p2)
    return (p2_01, p3_01, c_01)
#
def TupChain(collatzNumber):
    chain_path = ChainPath(collatzNumber)
    return TupChainFromPath(chain_path)
    return tup_chain
#

In [4]:
collatzChain(26)

[26, 13, 20, 10, 5, 8, 4, 2, 1]

In [5]:
collatzChain(51)

[51, 77, 116, 58, 29, 44, 22, 11, 17, 26, 13, 20, 10, 5, 8, 4, 2, 1]

In [6]:
# Not all Collatz Chains of 2n-1 include the chain of n:
collatzChain(9)

[9, 14, 7, 11, 17, 26, 13, 20, 10, 5, 8, 4, 2, 1]

In [7]:
collatzChain(17)

[17, 26, 13, 20, 10, 5, 8, 4, 2, 1]

In [8]:
collatzChain(12)

[12, 6, 3, 5, 8, 4, 2, 1]

In [9]:
collatzChain(23)

[23, 35, 53, 80, 40, 20, 10, 5, 8, 4, 2, 1]

## Because $x_{odd}\ \rightarrow\ \frac{3x_{odd} + 1}{2}$
All Odd paths start with 0 (down)

In [10]:
for x in range(5, 120):
    print("___ %d"%(x))
    print(ChainPath(x))
    print(ChainPath(2*x - 1))

___ 5
0111
0100010110111
___ 6
100111
0010110111
___ 7
00010110111
0110111
___ 8
111
000011110111
___ 9
0100010110111
010110111
___ 10
10111
00110010110111
___ 11
0010110111
011111
___ 12
1100111
00011110111
___ 13
0110111
0100110010110111
___ 14
100010110111
0010000010100100010000101100010010000001100001110101011101100011110111
___ 15
000011110111
0110010110111
___ 16
1111
0000010100100010000101100010010000001100001110101011101100011110111
___ 17
010110111
010100110010110111
___ 18
10100010110111
0011110111
___ 19
00110010110111
011100010110111
___ 20
110111
00010011100110010110111
___ 21
011111
010000010100100010000101100010010000001100001110101011101100011110111
___ 22
10010110111
00101011100010110111
___ 23
00011110111
011010110111
___ 24
11100111
000010100100010000101100010010000001100001110101011101100011110111
___ 25
0100110010110111
01011100010110111
___ 26
10110111
00110110010110111
___ 27
0010000010100100010000101100010010000001100001110101011101100011110111
011110111
___ 28


# Most Common Rule:
Seems to be prepend a down step and then change the first down following an up to an up

In [11]:
def rule1_isLegal(chain_path):
    if len(chain_path) <= 4:
        return False
    if "10" in chain_path[0:-4]:
        return True
    return False
#

def applyRule1(chain_path):
    chain_path = "0" + chain_path
    idx = chain_path.find("10")
    if idx == -1:
        return ""
    rule1_out = chain_path[0:idx+1] + "1" + chain_path[idx+2:]    
    return rule1_out
#
applyRule1("1010110111") == "01110110111"

True

In [13]:

rule1_nums = []
rule1_nums_1st_diff = []
contained_nums = []
for x in range(5, 512):
    chain_path = ChainPath(x)
    chain_path_rule1 = applyRule1(chain_path)
    chain_path_next = ChainPath(2*x - 1)
    annotate = ""
    if not rule1_isLegal(chain_path):
        annotate = "rule1_illegal"
    else:
        if chain_path_next == chain_path_rule1:
            annotate = "rule1"
            rule1_nums.append(x)
        else:
            if chain_path[0:-4] in chain_path_next[0:-4]:
                annotate = "contained"
                contained_nums.append(x)
    print("___ %d (%s)"%(x, annotate))
    print(chain_path)
    print(chain_path_next)
#
for i in range(len(rule1_nums)-1):
    rule1_nums_1st_diff.append(rule1_nums[i+1] - rule1_nums[i])

___ 5 (rule1_illegal)
0111
0100010110111
___ 6 (contained)
100111
0010110111
___ 7 ()
00010110111
0110111
___ 8 (rule1_illegal)
111
000011110111
___ 9 ()
0100010110111
010110111
___ 10 (rule1_illegal)
10111
00110010110111
___ 11 ()
0010110111
011111
___ 12 ()
1100111
00011110111
___ 13 (rule1_illegal)
0110111
0100110010110111
___ 14 ()
100010110111
0010000010100100010000101100010010000001100001110101011101100011110111
___ 15 (rule1_illegal)
000011110111
0110010110111
___ 16 (rule1_illegal)
1111
0000010100100010000101100010010000001100001110101011101100011110111
___ 17 (contained)
010110111
010100110010110111
___ 18 ()
10100010110111
0011110111
___ 19 ()
00110010110111
011100010110111
___ 20 (rule1_illegal)
110111
00010011100110010110111
___ 21 (rule1_illegal)
011111
010000010100100010000101100010010000001100001110101011101100011110111
___ 22 ()
10010110111
00101011100010110111
___ 23 (rule1_illegal)
00011110111
011010110111
___ 24 (contained)
11100111
0000101001000100001011000100100000

In [25]:
def updn_count(chain_path):
    dn = 0
    up = 0
    for i in range(len(chain_path)):
        if chain_path[i] == "0":
            dn += 1
        else:
            up += 1
    return (dn, up)
#
updn_count("010101110001011")

(7, 8)

In [ ]:
6,7,9,11,14,17,18,22,25,26,27,30,31

# Rule 1 seems to be related to https://oeis.org/A327175

The indexes of the zeros of: https://oeis.org/A327174 

$for\ \  r\ =\ \frac{1+\sqrt{5}}{2}$

$a(n) = floor((2n+1)r) - floor((n+1)r) - floor((n)r)$

... so is related to residuals of $\sqrt{5}$

In [36]:
for x in [14,17,18,22,25,26,27,30,31]:
    tup = TupChain(x)[-1]
    print("%d : %s"%(x, tup))

14 : (12, 5, 694)
17 : (9, 3, 53)
18 : (14, 6, 3262)
22 : (11, 4, 266)
25 : (16, 7, 10861)
26 : (8, 2, 22)
27 : (70, 41, 195820718533800070543)
30 : (13, 5, 902)
31 : (67, 39, 21944742846088148651)


In [38]:
rule1_nums_1st_diff

[1,
 2,
 2,
 3,
 3,
 1,
 4,
 3,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 3,
 1,
 1,
 3,
 3,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 3,
 1,
 4,
 3,
 1,
 1,
 3,
 3,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 3,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 3,
 1,
 4,
 3,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 3,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 3,
 1,
 4,
 3,
 1,
 1,
 3,
 3,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 3,
 1,
 1,
 3,
 3,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 3,
 1,
 4,
 3,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 3,
 1,
 1,
 3,
 3,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 3,
 1,
 4,
 3,
 1,
 1,
 3,
 3,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 3,
 1,
 1,
 3,
 3,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 3,
 1,
 4,
 3,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 3,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 3,
 1,
 4,
 3,
 1,
 1,
 3,
 3,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 1,
 2,
 1,
 4,
 3,
 1,
 1,


In [ ]:
1, 2, 2, 3, 3, 1, 4, 3, 1, 1, 3, 1, 2, 1, 4, 1, 2, 1, 1, 3, 1, 2, 1, 4

In [41]:
contained_nums

[12, 19, 24, 28, 29, 44, 52, 84, 168, 319, 325, 479, 488]